In [ ]:
import geopandas as gpd
import numpy as np
import shapely 
import matplotlib.pyplot as plt
import re
import glob
import os
%matplotlib widget

In [ ]:
SH=shapely.Polygon(np.c_[np.array([-180, 180, 180, -180]), np.array([-89, -89, -60,-60])])

track_re = re.compile('IS2_RGT_(\d+)_.*AA.kml')
tracks={'new':{}, 'old':{}}
for track_file in sorted(glob.glob('IS2_RGT*AA.kml')):
    print('----')
    print(track_file)
    layer_name = [name for name in gpd.list_layers(track_file).name if '.kml' in name][0]
    track=track_re.search(track_file).group(1)
    print(layer_name)
    tracks['new'][track]=gpd.read_file(track_file, layer=layer_name).set_crs(4326).clip(SH).to_crs(3031)
    old_track_file=glob.glob(f'assets/IS2_RGTs_cycle12_date_time/IS2_RGT_{track}*.kml')[0]
    layer_name = [name for name in gpd.list_layers(old_track_file).name if '.kml' in name][0]
    tracks['old'][track]=gpd.read_file(old_track_file, layer=layer_name).set_crs(4326).clip(SH).to_crs(3031)
    print(old_track_file)

In [ ]:
save_file='AA_ TestRGTs_RGT_transition_locations_V2.0.csv'
x_points=gpd.read_file(save_file)
for field in ['rgt','cycle','trans_type']:
    x_points[field]=[*map(int, x_points[field])]
for field in ['lon_req','lat_req']:
    x_points[field]=[*map(float, x_points[field])]
x_points['geometry'] = [shapely.Point([lon, lat]) for lon, lat in zip(x_points['lon_req'], x_points['lat_req'])]
x_points=gpd.GeoDataFrame(x_points)
x_points=x_points.set_crs(4326)
x_points=x_points.cx[:, -90:-50]
x_points=x_points.to_crs(3031)

In [ ]:
lat_dist_ot={}

for track in tracks['new']:
    geom_old=tracks['old'][track].geometry[0]
    geom_new=tracks['new'][track].geometry[0]
    geom_ll = tracks['new'][track].to_crs(4326).geometry[0]
    lat_dist_ot[track]=[]
    for xy, ll in zip(geom_new.coords, geom_ll.coords):
        pt = shapely.Point(xy)
        lat_dist_ot[track] += [[ll[1], geom_old.line_locate_point(pt),  geom_old.distance(pt)]]
    lat_dist_ot[track] = np.concatenate([lat_dist_ot[track]], axis=1)

In [ ]:
plt.figure()
for track in tracks['new'].keys():
    print(track)
    geom_old=tracks['old'][track].geometry[0]
    geom_new=tracks['new'][track].geometry[0]

    hl = plt.plot(*np.array(geom_old.coords)[:, 0:2].T,'--', label=track, linewidth=3)
    this_color=hl[0].get_color()
    plt.plot(*np.array(geom_new.coords)[:, 0:2].T, color=this_color, marker='.', label=track+' shifted')

    i_xp = x_points.rgt==int(track)
    if np.any(i_xp):
        temp= x_points[i_xp]
        plt.plot(temp.geometry.x, temp.geometry.y,'*', color=this_color, markersize=12)
    
plt.legend()
plt.gca().set_aspect(1)

In [ ]:
temp= x_points[i_xp].to_crs(3031)
temp

In [ ]:
plt.figure()
for track, this_ldo in lat_dist_ot.items():
    plt.plot(this_ldo[:,1]/1000, this_ldo[:,2], label=track, marker='.')
plt.legend()
plt.gca().set_xlabel('distance along track, km')
plt.gca().set_ylabel('off-tack offset, m')

In [ ]:
# send Christine the 200-km buffered Antarctica
# Talk to Tim about why there are different offsets on the tracks



In [ ]:
plt.figure()
for track, this_ldo in lat_dist_ot.items():
    plt.plot(this_ldo[:,0], this_ldo[:,2], label=track, marker='.')
plt.legend()
plt.gca().set_xlabel('latitude, ºN')
plt.gca().set_ylabel('off-tack offset, m')

In [ ]:
plt.figure()
for track, this_ldo in lat_dist_ot.items():
    plt.plot(this_ldo[:,0], this_ldo[:,2]/np.cos(this_ldo[:,0]*np.pi/180), label=track, marker='.')
plt.legend()
plt.gca().set_xlabel('latitude, ºN')
plt.gca().set_ylabel('off-tack offset/cos(lat), m')

In [ ]:
3441/np.cos(62*np.pi/180)

In [ ]:
plt.figure()
lat=np.arange(-88, -59)
plt.plot(lat, np.cos(lat*np.pi/180))

In [ ]:
! ls -lt *.csv

In [ ]:
x_points

In [ ]:
plt.figure()
plt.plot(x_points.geometry.x, x_points.geometry.y,'.')

In [ ]:
x_points[x_points.rgt==443+2*442]